In [1]:
import requests
import time
import random
import sqlite3
from tqdm import tqdm

# IP列表
'''
IP_LIST = ['20.213.247.195','13.74.59.33','145.40.121.101','45.167.125.97','157.100.12.138',
           '173.244.48.9','51.79.205.165','190.15.103.66','45.42.177.50','8.219.64.236']

def get_random_proxy():
    ip = random.choice(IP_LIST)
    return {"http": f"http://{ip}", "https": f"http://{ip}"}
'''

def init_db():
    conn = sqlite3.connect('web_info.db')  # 更改為 web_info.db
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS web_info (
            work_ID TEXT PRIMARY KEY,
            title TEXT,
            tags TEXT,
            like_count INTEGER,
            bookmark_count INTEGER,
            view_count INTEGER,
            image_url TEXT
        )''')
    conn.commit()
    return conn

def get_last_work_id(conn):
    c = conn.cursor()
    c.execute("SELECT MAX(work_ID) FROM web_info")
    result = c.fetchone()[0]
    return result if result else 0

connn = init_db()

start_id = int(get_last_work_id(connn)) + 1

In [2]:
import requests
import random
import time
from tqdm import tqdm

def check_pixiv_work_ids(start_id, max_attempts=1000):
    valid_ids = []
    
    # 創建tqdm進度條
    with tqdm(total=200, desc="Finding valid work IDs", unit="ID") as pbar:
        for attempt in range(max_attempts):
            work_id = start_id + attempt
            url = f"https://www.pixiv.net/artworks/{work_id}"
            
            try:
                #proxy = get_random_proxy()
                response = requests.head(url)
                if response.status_code == 200:
                    valid_ids.append(work_id)
                    pbar.update(1)  # 更新進度條
                    pbar.set_postfix_str(f"Latest ID: {work_id}")  # 顯示最新找到的ID
                
                if len(valid_ids) == 200:
                    break
                
                # 添加延遲以避免過於頻繁的請求
                delay = random.uniform(5, 15)  # 5到15秒的隨機延遲
                time.sleep(delay)
            
            except requests.RequestException as e:
                tqdm.write(f"Error checking work ID {work_id}: {e}")
    
    if len(valid_ids) == 200:
        print("\nSuccessfully found 50 valid work IDs.")
    else:
        print(f"\nReached maximum attempts. Found {len(valid_ids)} valid work IDs.")
    
    return valid_ids

# 使用函數
#start_id = 121188700  # 隨機選擇起始ID
print(f"Starting search from ID: {start_id}")
result = check_pixiv_work_ids(start_id)
print("Valid work IDs:", result)

Starting search from ID: 121416181


Finding valid work IDs:   0%|          | 0/200 [00:00<?, ?ID/s]

Finding valid work IDs:   0%|          | 1/200 [00:00<00:57,  3.45ID/s]

Finding valid work IDs:   0%|          | 1/200 [00:00<00:57,  3.45ID/s, Latest ID: 121416181]

Finding valid work IDs:   1%|          | 2/200 [00:06<12:55,  3.91s/ID, Latest ID: 121416181]

...

Finding valid work IDs: 100%|█████████▉| 199/200 [51:22<00:13, 13.63s/ID, Latest ID: 121416479]

Finding valid work IDs: 100%|██████████| 200/200 [51:30<00:00, 11.84s/ID, Latest ID: 121416479]

Finding valid work IDs: 100%|██████████| 200/200 [51:30<00:00, 11.84s/ID, Latest ID: 121416480]

Finding valid work IDs: 100%|██████████| 200/200 [51:30<00:00, 15.45s/ID, Latest ID: 121416480]


Successfully found 50 valid work IDs.
Valid work IDs: [121416181, 121416182, 121416185, 121416186, 121416187, 121416188, 121416193, 121416196, 121416197, 121416198, 121416201, 121416203, 121416204, 121416206, 121416207, 121416209, 121416210, 121416211, 121416212, 121416214, 121416217, 121416219, 121416220, 121416221, 121416222, 121416224, 121416226, 121416229, 121416230, 121416232, 121416233, 121416234, 121416236, 121416238, 121416239, 121416240, 121416241, 121416242, 121416243, 121416246, 121416248, 121416249, 121416250, 121416252, 121416254, 121416255, 121416256, 121416257, 121416258, 121416260, 121416264, 121416267, 121416268, 121416269, 121416271, 121416274, 121416275, 121416276, 121416277, 121416278, 121416279, 121416280, 121416282, 121416283, 121416285, 121416286, 121416287, 121416289, 121416290, 121416291, 121416292, 121416293, 121416294, 121416300, 121416301, 121416302, 121416303, 121416304, 121416305, 121416306, 121416308, 121416309, 121416310, 121416312, 121416313, 121416315

In [3]:
import requests
from bs4 import BeautifulSoup
import os

def save_webpage_as_single_file(url, filename):
    try:
        # 发送 GET 请求获取网页内容
        response = requests.get(url)
        response.raise_for_status()  # 检查请求是否成功
        #time.sleep(np.random.randint(3,7))

        # 解析网页内容
        soup = BeautifulSoup(response.text, 'html.parser')

        # 获取网页的标题
        title = soup.title.string if soup.title else 'webpage'
        
        # 创建 .mhtml 文件内容
        mhtml_content = f"<!DOCTYPE html>\n<html>\n<head>\n<title>{title}</title>\n</head>\n<body>\n"
        mhtml_content += str(soup)  # 添加网页内容
        mhtml_content += "\n</body>\n</html>"

        # 保存为 .mhtml 文件
        with open(filename, 'w', encoding='utf-8') as file:
            file.write(mhtml_content)

        print(f"网页内容已成功保存为 {filename}")
    except requests.exceptions.RequestException as e:
        print(f"请求失败: {e}")
        pass
    except Exception as e:
        print(f"保存文件时出错: {e}")

# 输入 URL 和文件名




In [4]:
import os
import sqlite3
import re

# 提取信息的函数
def extract_info_from_mhtml(mhtml_file,work_ID):
    with open(mhtml_file, 'r', encoding='utf-8') as file:
        content = file.read()
    
    # 使用 BeautifulSoup 解析 HTML 内容
    soup = BeautifulSoup(content, 'html.parser')

    # 提取 <title> 标签的内容
    title = soup.title.string if soup.title else '无标题'
    
     # 使用正则表达式提取多个 "tag"
    tag_pattern = re.findall(r'"tag"\s*:\s*"([^"]+)"', content)
    
    # 使用正则表达式提取 "likeCount", "bookmarkCount", "viewCount"
    like_count_pattern = re.search(r'"likeCount"\s*:\s*(\d+)', content)
    bookmark_count_pattern = re.search(r'"bookmarkCount"\s*:\s*(\d+)', content)
    view_count_pattern = re.search(r'"viewCount"\s*:\s*(\d+)', content)
    image_pattern = re.search(r'"regular"\s*:\s*"([^"]+)"', content)

    # 获取正则表达式的匹配结果
    tags = tag_pattern if tag_pattern else ['无标签']
    like_count = like_count_pattern.group(1) if like_count_pattern else '无点赞数'
    bookmark_count = bookmark_count_pattern.group(1) if bookmark_count_pattern else '无收藏数'
    view_count = view_count_pattern.group(1) if view_count_pattern else '无浏览数'
    image_count = image_pattern.group(1) if image_pattern else '無影像連結'
    return {
        'work_ID': str(work_ID),
        'title': title.split(' - ')[0],  # 清理标题内容
        'tags': ', '.join(tags),  # 标签列表转为字符串
        'like_count': like_count,
        'bookmark_count': bookmark_count,
        'view_count': view_count,
        'image url': image_count
        
    }



# 将数据追加到 SQLite 数据库的函数
def append_to_sqlite(info, db_filename='web_info.db'):
    # 连接 SQLite 数据库（如果文件不存在，会自动创建）
    conn = sqlite3.connect(db_filename)
    cursor = conn.cursor()

    # 创建表格（如果不存在）
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS web_info (
            work_ID TEXT PRIMARY KEY,
            title TEXT,
            tags TEXT,
            like_count INTEGER,
            bookmark_count INTEGER,
            view_count INTEGER,
            image_url TEXT
        )
    ''')

    # 插入数据，如果 work_ID 已存在则忽略插入
    cursor.execute('''
        INSERT OR REPLACE INTO web_info (work_ID, title, tags, like_count, bookmark_count, view_count, image_url)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    ''', (
        info['work_ID'],
        info['title'],
        info['tags'],
        int(info['like_count']),
        int(info['bookmark_count']),
        int(info['view_count']),
        info['image url']
    ))

    # 提交更改并关闭连接
    conn.commit()
    conn.close()





In [5]:
db_filename = 'web_info.db'
for work_ID in result:
    url = f'https://www.pixiv.net/artworks/{work_ID}'
    filename = f'{work_ID}.mhtml'
    try:
        save_webpage_as_single_file(url, filename)
        # 示例：提取和追加数据
        mhtml_file = f'{work_ID}.mhtml'  # 替换为实际的 .mhtml 文件路径
        info = extract_info_from_mhtml(mhtml_file,work_ID)
        # 将提取的信息追加到 'web_info.xlsx'
        append_to_sqlite(info)
        print(f"新信息已成功追加到 {db_filename}")
    except Exception as e:
        pass
    
    delay = int(random.uniform(30, 60))  # 5到15秒的隨機延遲
    print(f"休息 {delay} 秒鐘")
    time.sleep(delay)
    



网页内容已成功保存为 121416181.mhtml
休息 57 秒鐘


网页内容已成功保存为 121416182.mhtml
休息 45 秒鐘


网页内容已成功保存为 121416185.mhtml
休息 41 秒鐘


网页内容已成功保存为 121416186.mhtml
休息 41 秒鐘


网页内容已成功保存为 121416187.mhtml
休息 46 秒鐘


...

网页内容已成功保存为 121416475.mhtml
休息 44 秒鐘


网页内容已成功保存为 121416476.mhtml
休息 55 秒鐘


网页内容已成功保存为 121416477.mhtml
休息 36 秒鐘


网页内容已成功保存为 121416479.mhtml
休息 30 秒鐘


网页内容已成功保存为 121416480.mhtml
休息 39 秒鐘


In [6]:
import sqlite3

# 連接到 SQLite 資料庫
con = sqlite3.connect('web_info.db')
cur = con.cursor()

# 使用 SQL 查詢計算 work_ID 的總數
cur.execute("SELECT COUNT(work_ID) FROM web_info")
work_id_count = cur.fetchone()[0]

# 顯示 work_ID 總數
print(f"Total number of work_IDs: {work_id_count}")

# 關閉資料庫連線
con.close()


Total number of work_IDs: 178059


In [7]:
'''
import requests

# 图片的 URL
url = 'https://i.pximg.net/img-master/img/2024/08/05/22/47/33/121215791_p0_master1200.jpg'

# 图片保存的本地文件名
filename = 'downloaded_image.png'

# 请求头，模拟完整的浏览器请求
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
    'Referer': 'https://www.pixiv.net/',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
    'Connection': 'keep-alive'
}

# 发起 GET 请求以下载图片
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    # 将图片数据写入本地文件
    with open(filename, 'wb') as file:
        file.write(response.content)
    print(f"图片已成功保存为 {filename}")
else:
    print(f"图片下载失败，状态码：{response.status_code}")

'''


'\nimport requests\n\n# 图片的 URL\nurl = \'https://i.pximg.net/img-master/img/2024/08/05/22/47/33/121215791_p0_master1200.jpg\'\n\n# 图片保存的本地文件名\nfilename = \'downloaded_image.png\'\n\n# 请求头，模拟完整的浏览器请求\nheaders = {\n    \'User-Agent\': \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36\',\n    \'Referer\': \'https://www.pixiv.net/\',\n    \'Accept\': \'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8\',\n    \'Accept-Encoding\': \'gzip, deflate, br\',\n    \'Accept-Language\': \'en-US,en;q=0.9\',\n    \'Connection\': \'keep-alive\'\n}\n\n# 发起 GET 请求以下载图片\nresponse = requests.get(url, headers=headers)\n\n# 检查请求是否成功\nif response.status_code == 200:\n    # 将图片数据写入本地文件\n    with open(filename, \'wb\') as file:\n        file.write(response.content)\n    print(f"图片已成功保存为 {filename}")\nelse:\n    print(f"图片下载失败，状态码：{response.status_code}")\n\n'